In [6]:
VIDEO_PATH = './data/scene-2.mov'
SAVE_DIR = "./data/processed/scene-2/"
VIDEO_OUT = './data/processed/scene_2.mp4'
module_handle = "https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1"

In [1]:
import cv2
import plotly.express as px
import os
from tqdm.notebook import tqdm
import tensorflow_hub as hub
import tensorflow as tf
from PIL import Image

In [2]:
# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

In [3]:
def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color,
                               font,
                               thickness=4,
                               display_str_list=()):
  """Adds a bounding box to an image."""
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
  draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

  # If the total height of the display strings added to the top of the bounding
  # box exceeds the top of the image, stack the strings below the bounding box
  # instead of above.
  display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = top + total_display_str_height
  # Reverse list and print from bottom to top.
  for display_str in display_str_list[::-1]:
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                    (left + text_width, text_bottom)],
                   fill=color)
    draw.text((left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
    text_bottom -= text_height - 2 * margin


def draw_boxes(image, boxes, class_ids, class_names, scores, font, max_boxes=10, min_score=0.1):
  """Overlay labeled boxes on an image with formatted scores and label names."""
  colors = list(ImageColor.colormap.values())

  for i in range(min(boxes.shape[0], max_boxes)):
    if scores[i] >= min_score:
      ymin, xmin, ymax, xmax = tuple(boxes[i])
      display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                     int(100 * scores[i]))
      color = colors[class_ids[i] % len(colors)]
      image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
      draw_bounding_box_on_image(
          image_pil,
          ymin,
          xmin,
          ymax,
          xmax,
          color,
          font,
          display_str_list=[display_str])
      np.copyto(image, np.array(image_pil))
  return np.array(image_pil)

In [4]:
def fast_draw_boxes(image, boxes, class_ids, class_names, scores, font, max_boxes=10, min_score=0.1):
  """Overlay labeled boxes on an image with formatted scores and label names."""
  colors = list(ImageColor.colormap.values())
  image_pil = Image.fromarray(np.uint8(image)).convert("RGB")

  for i in range(min(boxes.shape[0], max_boxes)):
    if scores[i] >= min_score:
      ymin, xmin, ymax, xmax = tuple(boxes[i])
      display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                     int(100 * scores[i]))
      color = colors[class_ids[i] % len(colors)]
      draw_bounding_box_on_image(
          image_pil,
          ymin,
          xmin,
          ymax,
          xmax,
          color,
          font,
          display_str_list=[display_str])
#       np.copyto(image, np.array(image_pil))
  return np.array(image_pil)

In [5]:
%%time

frames = []

cap = cv2.VideoCapture(VIDEO_PATH)
ret = True

while ret:
    ret, frame = cap.read()
    if ret:
        frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
        frames.append(frame)

CPU times: user 8.07 s, sys: 659 ms, total: 8.73 s
Wall time: 3.91 s


In [6]:
detector = hub.load(module_handle).signatures['default']

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [7]:
try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                          25)
except IOError:
    print("Font not found, using default font.")
    font = ImageFont.load_default()

In [8]:
processed_frames = []
sample_rate = 3

for i, img in enumerate(tqdm(frames)):
    if i % sample_rate == 0:
        resized = cv2.resize(img, (512, 512))
        image_tensor = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]

        result = detector(image_tensor)

    image_with_boxes = fast_draw_boxes(
        img.copy(), 
        result["detection_boxes"].numpy(),
        result['detection_class_labels'].numpy(),
        result["detection_class_entities"].numpy(), 
        result["detection_scores"].numpy(),
        font=font
    )
    
    processed_frames.append(image_with_boxes)

NameError: name 'frames' is not defined

In [122]:
os.makedirs(SAVE_DIR, exist_ok=True)
for i, frame in enumerate(tqdm(processed_frames)):
    cv2.imwrite(os.path.join(SAVE_DIR, f"frame-{i}.jpg"), frame, [cv2.IMWRITE_JPEG_QUALITY, 90])


CPU times: user 17 s, sys: 365 ms, total: 17.4 s
Wall time: 17.4 s


In [139]:
# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter(VIDEO_OUT,fourcc, 30, (1280, 720))

for frame in tqdm(processed_frames):
    out.write(frame)

out.release()